### Bayesian network tutorial powered by pgmpy
conda install -c ankurankan pgmpy
pip install jupyter
pip install -U scikit-learn

In [ ]:
# Importing Library
from pgmpy.models import BayesianNetwork
from pgmpy.inference import VariableElimination

alarm_model = BayesianNetwork(
    [
        ("Burglary", "Alarm"),
        ("Earthquake", "Alarm"),
        ("Alarm", "JohnCalls"),
        ("Alarm", "MaryCalls"),
    ]
)

In [ ]:
# Defining the parameters using CPT
from pgmpy.factors.discrete import TabularCPD

cpd_burglary = TabularCPD(
    variable="Burglary", variable_card=2, values=[[0.999], [0.001]]
)
cpd_earthquake = TabularCPD(
    variable="Earthquake", variable_card=2, values=[[0.998], [0.002]]
)
cpd_alarm = TabularCPD(
    variable="Alarm",
    variable_card=2,
    values=[[0.999, 0.71, 0.06, 0.05], [0.001, 0.29, 0.94, 0.95]],
    evidence=["Burglary", "Earthquake"],
    evidence_card=[2, 2],
)
cpd_johncalls = TabularCPD(
    variable="JohnCalls",
    variable_card=2,
    values=[[0.95, 0.1], [0.05, 0.9]],
    evidence=["Alarm"],
    evidence_card=[2],
)
cpd_marycalls = TabularCPD(
    variable="MaryCalls",
    variable_card=2,
    values=[[0.1, 0.7], [0.9, 0.3]],
    evidence=["Alarm"],
    evidence_card=[2],
)

In [ ]:
# Associating the parameters with the model structure
alarm_model.add_cpds(
    cpd_burglary, cpd_earthquake, cpd_alarm, cpd_johncalls, cpd_marycalls
)

In [ ]:
# Checking if the cpds are valid for the model
alarm_model.check_model()

In [ ]:
# Viewing nodes of the model
alarm_model.nodes()

In [ ]:
# Viewing edges of the model
alarm_model.edges()

In [ ]:
# Checking independcies of a node
alarm_model.local_independencies("Burglary")

In [ ]:
# Listing all Independencies
alarm_model.get_independencies()

### Parameter Learning in Discrete Bayesian Networks

In this notebook, we show an example for learning the parameters (CPDs) of a Discrete Bayesian Network given the data and the model structure. pgmpy has two main methods for learning the parameters:

* MaximumLikelihood Estimator (pgmpy.estimators.MaximumLikelihoodEstimator)
* Bayesian Estimator (pgmpy.estimators.BayesianEstimator)
* Expectation Maximization (pgmpy.estimators.ExpectationMaximization)
In the examples, we will try to generate some data from given models and then try to learn the model parameters back from the generated data.

In [16]:
# Step 1: Generate some data
# Use the alarm model to generate data from it.

from pgmpy.utils import get_example_model
from pgmpy.sampling import BayesianModelSampling
import numpy as np
alarm_model = get_example_model("alarm")
samples = BayesianModelSampling(alarm_model).forward_sample(size=int(1e5))
samples.head()

  0%|          | 0/37 [00:00<?, ?it/s]

,HISTORY,CVP,PCWP,HYPOVOLEMIA,LVEDVOLUME,LVFAILURE,STROKEVOLUME,ERRLOWOUTPUT,HRBP,HREKG,...,MINVOLSET,VENTMACH,VENTTUBE,VENTLUNG,VENTALV,ARTCO2,CATECHOL,HR,CO,BP
0,FALSE,NORMAL,NORMAL,FALSE,NORMAL,FALSE,NORMAL,FALSE,HIGH,HIGH,...,NORMAL,NORMAL,ZERO,LOW,HIGH,LOW,HIGH,HIGH,HIGH,LOW
1,FALSE,NORMAL,NORMAL,FALSE,NORMAL,FALSE,NORMAL,FALSE,HIGH,HIGH,...,NORMAL,NORMAL,LOW,LOW,HIGH,LOW,HIGH,HIGH,HIGH,HIGH
2,FALSE,NORMAL,NORMAL,FALSE,NORMAL,FALSE,NORMAL,FALSE,HIGH,HIGH,...,NORMAL,ZERO,ZERO,ZERO,ZERO,HIGH,HIGH,HIGH,HIGH,HIGH
3,FALSE,HIGH,HIGH,TRUE,HIGH,FALSE,NORMAL,TRUE,NORMAL,LOW,...,NORMAL,NORMAL,LOW,ZERO,ZERO,HIGH,NORMAL,NORMAL,NORMAL,NORMAL
4,FALSE,NORMAL,NORMAL,TRUE,NORMAL,FALSE,NORMAL,FALSE,HIGH,HIGH,...,NORMAL,NORMAL,LOW,ZERO,ZERO,HIGH,HIGH,HIGH,HIGH,LOW


In [17]:
# Step 2: Define a model structure
# Defining the Bayesian Model structure

from pgmpy.models import BayesianNetwork

model_struct = BayesianNetwork(ebunch=alarm_model.edges())
model_struct.nodes()

NodeView(('HYPOVOLEMIA', 'LVEDVOLUME', 'STROKEVOLUME', 'CVP', 'PCWP', 'LVFAILURE', 'HISTORY', 'CO', 'ERRLOWOUTPUT', 'HRBP', 'ERRCAUTER', 'HREKG', 'HRSAT', 'INSUFFANESTH', 'CATECHOL', 'ANAPHYLAXIS', 'TPR', 'BP', 'KINKEDTUBE', 'PRESS', 'VENTLUNG', 'FIO2', 'PVSAT', 'SAO2', 'PULMEMBOLUS', 'PAP', 'SHUNT', 'INTUBATION', 'MINVOL', 'VENTALV', 'DISCONNECT', 'VENTTUBE', 'MINVOLSET', 'VENTMACH', 'EXPCO2', 'ARTCO2', 'HR'))

In [18]:
# Step 3: Learning the model parameters
# Fitting the model using Maximum Likelihood Estimator

from pgmpy.estimators import MaximumLikelihoodEstimator

mle = MaximumLikelihoodEstimator(model=model_struct, data=samples)

# Estimating the CPD for a single node.
print(mle.estimate_cpd(node="FIO2"))
print(mle.estimate_cpd(node="CVP"))

# Estimating CPDs for all the nodes in the model
mle.get_parameters()[:10]  # Show just the first 10 CPDs in the output

+--------------+------+
| FIO2(LOW)    | 0.05 |
+--------------+------+
| FIO2(NORMAL) | 0.95 |
+--------------+------+
+-------------+-----+----------------------+
| LVEDVOLUME  | ... | LVEDVOLUME(NORMAL)   |
+-------------+-----+----------------------+
| CVP(HIGH)   | ... | 0.009998862149399783 |
+-------------+-----+----------------------+
| CVP(LOW)    | ... | 0.03971098594754509  |
+-------------+-----+----------------------+
| CVP(NORMAL) | ... | 0.9502901519030551   |
+-------------+-----+----------------------+


[<TabularCPD representing P(HYPOVOLEMIA:2) at 0x7f1c092b0b90>,
 <TabularCPD representing P(LVEDVOLUME:3 | HYPOVOLEMIA:2, LVFAILURE:2) at 0x7f1c092b54d0>,
 <TabularCPD representing P(STROKEVOLUME:3 | HYPOVOLEMIA:2, LVFAILURE:2) at 0x7f1c092b5e50>,
 <TabularCPD representing P(CVP:3 | LVEDVOLUME:3) at 0x7f1c092b0290>,
 <TabularCPD representing P(PCWP:3 | LVEDVOLUME:3) at 0x7f1c093c9390>,
 <TabularCPD representing P(LVFAILURE:2) at 0x7f1c0929fd90>,
 <TabularCPD representing P(HISTORY:2 | LVFAILURE:2) at 0x7f1c0929fb10>,
 <TabularCPD representing P(CO:3 | HR:3, STROKEVOLUME:3) at 0x7f1c08a57650>,
 <TabularCPD representing P(ERRLOWOUTPUT:2) at 0x7f1c092a9d90>,
 <TabularCPD representing P(HRBP:3 | ERRLOWOUTPUT:2, HR:3) at 0x7f1c08a60190>]

In [19]:
# Verifying that the learned parameters are almost equal.
np.allclose(
    alarm_model.get_cpds("FIO2").values, mle.estimate_cpd("FIO2").values, atol=0.01
)

True

In [20]:
# Fitting the using Bayesian Estimator
from pgmpy.estimators import BayesianEstimator

best = BayesianEstimator(model=model_struct, data=samples)

print(best.estimate_cpd(node="FIO2", prior_type="BDeu", equivalent_sample_size=1000))
# Uniform pseudo count for each state. Can also accept an array of the size of CPD.
print(best.estimate_cpd(node="CVP", prior_type="dirichlet", pseudo_counts=100))

# Learning CPDs for all the nodes in the model. For learning all parameters with BDeU prior, a dict of
# pseudo_counts need to be provided
best.get_parameters(prior_type="BDeu", equivalent_sample_size=1000)[:10]

+--------------+-----------+
| FIO2(LOW)    | 0.0544554 |
+--------------+-----------+
| FIO2(NORMAL) | 0.945545  |
+--------------+-----------+
+-------------+-----+----------------------+
| LVEDVOLUME  | ... | LVEDVOLUME(NORMAL)   |
+-------------+-----+----------------------+
| CVP(HIGH)   | ... | 0.011372648991615681 |
+-------------+-----+----------------------+
| CVP(LOW)    | ... | 0.04095853161114888  |
+-------------+-----+----------------------+
| CVP(NORMAL) | ... | 0.9476688193972355   |
+-------------+-----+----------------------+


[<TabularCPD representing P(HYPOVOLEMIA:2) at 0x7f1c08a0e8d0>,
 <TabularCPD representing P(LVEDVOLUME:3 | HYPOVOLEMIA:2, LVFAILURE:2) at 0x7f1c08a77910>,
 <TabularCPD representing P(STROKEVOLUME:3 | HYPOVOLEMIA:2, LVFAILURE:2) at 0x7f1c08a81b90>,
 <TabularCPD representing P(CVP:3 | LVEDVOLUME:3) at 0x7f1c08a60b50>,
 <TabularCPD representing P(PCWP:3 | LVEDVOLUME:3) at 0x7f1c08a77fd0>,
 <TabularCPD representing P(LVFAILURE:2) at 0x7f1c08a7fd10>,
 <TabularCPD representing P(HISTORY:2 | LVFAILURE:2) at 0x7f1c08a7d510>,
 <TabularCPD representing P(CO:3 | HR:3, STROKEVOLUME:3) at 0x7f1c08a18e50>,
 <TabularCPD representing P(ERRLOWOUTPUT:2) at 0x7f1c08a7f1d0>,
 <TabularCPD representing P(HRBP:3 | ERRLOWOUTPUT:2, HR:3) at 0x7f1c08a778d0>]

In [21]:
# Shortcut for learning all the parameters and adding the CPDs to the model.

model_struct = BayesianNetwork(ebunch=alarm_model.edges())
model_struct.fit(data=samples, estimator=MaximumLikelihoodEstimator)
print(model_struct.get_cpds("FIO2"))

model_struct = BayesianNetwork(ebunch=alarm_model.edges())
model_struct.fit(
    data=samples,
    estimator=BayesianEstimator,
    prior_type="BDeu",
    equivalent_sample_size=1000,
)
print(model_struct.get_cpds("FIO2"))

+--------------+------+
| FIO2(LOW)    | 0.05 |
+--------------+------+
| FIO2(NORMAL) | 0.95 |
+--------------+------+
+--------------+-----------+
| FIO2(LOW)    | 0.0544554 |
+--------------+-----------+
| FIO2(NORMAL) | 0.945545  |
+--------------+-----------+


In [22]:
from pgmpy.estimators import ExpectationMaximization as EM

# Define a model structure with latent variables
model_latent = BayesianNetwork(
    ebunch=alarm_model.edges(), latents=["HYPOVOLEMIA", "LVEDVOLUME", "STROKEVOLUME"]
)

# Dataset for latent model which doesn't have values for the latent variables
samples_latent = samples.drop(model_latent.latents, axis=1)

model_latent.fit(samples_latent, estimator=EM)

  0%|          | 0/100 [00:00<?, ?it/s]

### Inference in Discrete Bayesian Network

In this notebook, we show a simple example for doing Exact inference in Bayesian Networks using pgmpy. We will be using the Asia network (http://www.bnlearn.com/bnrepository/#asia) for this example.

Step 1: Define the model.

In [1]:
# Fetch the asia model from the bnlearn repository

from pgmpy.utils import get_example_model
from pgmpy.sampling import BayesianModelSampling
from pgmpy.factors.discrete import TabularCPD

asia_model = get_example_model("asia")
samples = BayesianModelSampling(asia_model).forward_sample(size=int(1e5))
samples.head()

  0%|          | 0/8 [00:00<?, ?it/s]

,asia,tub,smoke,lung,bronc,either,xray,dysp
0,no,no,no,no,no,no,no,no
1,no,no,no,no,no,no,no,no
2,no,no,no,no,no,no,no,no
3,no,no,yes,no,no,no,no,no
4,no,no,yes,no,no,no,no,no


In [7]:
print("Nodes: ", asia_model.nodes())
print("Edges: ", asia_model.edges())
asia_model.get_cpds()

Nodes:  ['asia', 'tub', 'smoke', 'lung', 'bronc', 'either', 'xray', 'dysp']
Edges:  [('asia', 'tub'), ('tub', 'either'), ('smoke', 'lung'), ('smoke', 'bronc'), ('lung', 'either'), ('bronc', 'dysp'), ('either', 'xray'), ('either', 'dysp')]


[<TabularCPD representing P(asia:2) at 0x7f1c09e07810>,
 <TabularCPD representing P(bronc:2 | smoke:2) at 0x7f1c09dfee50>,
 <TabularCPD representing P(dysp:2 | bronc:2, either:2) at 0x7f1c09dfe8d0>,
 <TabularCPD representing P(either:2 | lung:2, tub:2) at 0x7f1c09dfea90>,
 <TabularCPD representing P(lung:2 | smoke:2) at 0x7f1c09dfe650>,
 <TabularCPD representing P(smoke:2) at 0x7f1c09dfe210>,
 <TabularCPD representing P(tub:2 | asia:2) at 0x7f1c09dfe890>,
 <TabularCPD representing P(xray:2 | either:2) at 0x7f1c09d91cd0>]

### Step 2: Initialize the inference class

Currently, pgmpy support two algorithms for inference: 1. Variable Elimination(변수 축소) and, 2. Belief Propagation(신뢰 전파). Both of these are exact inferece algorithms. The following example uses VariableElimination but BeliefPropagation has an identifcal API, so all the methods show below would also work for BeliefPropagation.

In [8]:
# Initializing the VariableElimination class

from pgmpy.inference import VariableElimination

asia_infer = VariableElimination(asia_model)

### Step 3: Doing Inference using hard evidence

In [9]:
# Computing the probability of bronc given smoke=no.
q = asia_infer.query(variables=["bronc"], evidence={"smoke": "no"})
print(q)

# Computing the joint probability of bronc and asia given smoke=yes
q = asia_infer.query(variables=["bronc", "asia"], evidence={"smoke": "yes"})
print(q)

# Computing the probabilities (not joint) of bronc and asia given smoke=no
q = asia_infer.query(variables=["bronc", "asia"], evidence={"smoke": "no"}, joint=False)
for factor in q.values():
    print(factor)

0it [00:00, ?it/s]

0it [00:00, ?it/s]

+------------+--------------+
| bronc      |   phi(bronc) |
+============+==============+
| bronc(yes) |       0.3000 |
+------------+--------------+
| bronc(no)  |       0.7000 |
+------------+--------------+


0it [00:00, ?it/s]

0it [00:00, ?it/s]

+------------+-----------+-------------------+
| bronc      | asia      |   phi(bronc,asia) |
+============+===========+===================+
| bronc(yes) | asia(yes) |            0.0060 |
+------------+-----------+-------------------+
| bronc(yes) | asia(no)  |            0.5940 |
+------------+-----------+-------------------+
| bronc(no)  | asia(yes) |            0.0040 |
+------------+-----------+-------------------+
| bronc(no)  | asia(no)  |            0.3960 |
+------------+-----------+-------------------+


0it [00:00, ?it/s]

0it [00:00, ?it/s]

+------------+--------------+
| bronc      |   phi(bronc) |
+============+==============+
| bronc(yes) |       0.3000 |
+------------+--------------+
| bronc(no)  |       0.7000 |
+------------+--------------+
+-----------+-------------+
| asia      |   phi(asia) |
+===========+=============+
| asia(yes) |      0.0100 |
+-----------+-------------+
| asia(no)  |      0.9900 |
+-----------+-------------+


In [10]:
# Computing the MAP of bronc given smoke=no.
q = asia_infer.map_query(variables=["bronc"], evidence={"smoke": "no"})
print(q)

# Computing the MAP of bronc and asia given smoke=yes
q = asia_infer.map_query(variables=["bronc", "asia"], evidence={"smoke": "yes"})
print(q)

0it [00:00, ?it/s]

0it [00:00, ?it/s]

{'bronc': 'no'}


0it [00:00, ?it/s]

0it [00:00, ?it/s]

{'bronc': 'yes', 'asia': 'no'}
